In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import os
from scipy import stats

In [ ]:
def ReadData(fileLocation):
    #read data in relation to format
    try:
        if fileLocation.endswith('.csv'): 
            dataRead =pd.DataFrame(pd.read_csv(fileLocation,delimiter = " "))
            dataRead.columns = ['X','Y','Z']
        elif fileLocation.endswith('.tsv'): 
            dataRead = np.genfromtxt(fileLocation, delimiter = " " )
            x = dataRead[:,0]
            y= dataRead[:,1]
            z =dataRead[:,2]
            dataRead = pd.DataFrame({'X':x,'Y':y, 'Z':z})
        elif fileLocation.endswith('.xls'):
            dataRead = pd.read_excel(fileLocation)
        elif fileLocation.endswith('.txt'):
            dataRead =pd.read_csv(fileLocation, sep=' ')
            dataRead.columns = ['X','Y','Z']
           
        else:
            # Unsupported file format
            raise ValueError(f"Unsupported file format: {fileLocation}")
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

    return dataRead



In [ ]:
#looking for the max directly
def DirectMaxSearch(Voltage):
    maxIndex = np.argmax(Voltage)
    return maxIndex

In [23]:
#Looking for max using the guassian fit
def gaussian_3d(x, y, a, mu_x, mu_y, sigma_x, sigma_y):
    """Calculates the 3D Gaussian function value for given x and y."""
    return a * np.exp(-((x - mu_x)**2 / (2 * sigma_x**2) + (y - mu_y)**2 / (2 * sigma_y**2)))

def fit_gaussian_3d(x, y, z):
    """Fits a 3D Gaussian curve to the given data points and finds the maximum position.

    Args:
        x: The x-axis data points.
        y: The y-axis data points.
        z: The z-axis data points.

    Returns:
        A tuple containing the x and y positions of the maximum value.
    """

    # Preprocessing (smoothing)
    z_smoothed = np.convolve(z, np.ones(5) / 5, mode='same')

    # Improved initial guess
    a = np.max(z_smoothed)
    p0 = [a, np.mean(x), np.mean(y), np.std(x), np.std(y)]
    try:
        popt, pcov = curve_fit(gaussian_3d, x, y, z_smoothed, p0=p0)
        peak_gaussian_x = popt[1]  # Extract the x-position of the maximum
        peak_gaussian_y = popt[2]  # Extract the y-position of the maximum
        return (peak_gaussian_x, peak_gaussian_y)
    except RuntimeError:
        print("Curve fitting failed")
        return None

In [24]:
# function to analyse and plot the dataset
def analyze_dataset(dataset,count,index):
    voltage = dataset["Z"]
    y_value = dataset["Y"]
    x_value = dataset["X"]
    #direct Max analysis
    dir_max_index = DirectMaxSearch(voltage)
    dir_max_posX = x_value[dir_max_index]
    dir_max_posY=y_value[dir_max_index]
   
    
    dir_max_val=np.max(voltage)
    dir_max_data ={dir_max_posX,dir_max_posY,dir_max_val}
    print(f"For this dataset, direct max X position is: {dir_max_posX} direct max Y position is: {dir_max_posY} and value is {dir_max_val}")
    
    #gaussian max analysis is 
    gauss_max_posX,guass_max_posY = fit_gaussian_3d(x_value,y_value,voltage)
    if gauss_max_posX and gauss_max_posY is not None:
        gauss_max_val=voltage[np.argmax(gauss_max_posX)]
        gauss_max_data ={gauss_max_posX,gaus_max_posY,gauss_max_val}
        print(f"For this dataset, Gaussian max position is: {gauss_max_pos} and value is {gauss_max_val}")
    else:
        print("Gaussian fit failed for this dataset.")
    #plot the data
    plt.figure()
    ax =plt.axes(projection='3d')
    ax.scatter(x_value, y_value, voltage)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Voltage")
    ax.set_title(count + str(index))
    plt.show()
   
    return (dir_max_data, gauss_max_data)

In [20]:
#declare 2 arrays that will hold the data from the anaylyse_dataset function
dir_results = []
gauss_results =[]
directory_path ="3D_data"
index = 1 #to help identify the index of the file we are currently processing

#loop through the data set folder and perform all the necessary operations(read the data, process the maxima etc)

for file_name in os.listdir(directory_path): #get a single dataset at a time (identified by file_name)
    file_path = os.path.join(directory_path, file_name)
    data = ReadData(file_path)
    if data is not None:
        direct, gauss = analyze_dataset(data,file_name,index) #analyse dataset and return a tuple containind the direct max data and the gaussian max data
       
        #add the results to an array containing the results from other datasets
        dir_results.append(direct)
        gauss_results.append(gauss)
        index +=1


For this dataset, direct max X position is: 11.740355 direct max Y position is: 11.12429 and value is 0.1880720311253


TypeError: curve_fit() got multiple values for argument 'p0'

In [ ]:
#storing the data gotten from reading the datasets in seperate files

df_gauss=pd.DataFrame(gauss_results) 
df_gauss.to_csv("gaussianData.csv", index =False) #store guassian max data in gaussianData.csv file

df_dir=pd.DataFrame(dir_results)
df_dir.to_csv("directData.csv", index =False) #store the direct max data in directData.csv

In [ ]:
#Plotting the final maximum data and their positions
def read_and_process_final_datasets(fileName):
    df =pd.DataFrame(pd.read_csv(fileName, delimiter =",")) #read the data from the file where we stored it after processing all the datasets
    df.columns =["x","y","z"]
    volt =df["z"] 
    y_val =df["y"]
    x_val = df["x"]
    #plot the data
   plt.figure()
    ax =plt.axes(projection='3d')
    ax.scatter(x_value, y_value, voltage)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Voltage")
    ax.set_title(count + str(index))
    plt.grid()
    plt.show()
    
    #displaying the best fitline graph
    # slope, intercept, r_value, p_value, std_err = stats.linregress(x_val, volt)
    
    # # Create the best fit line function
    # def best_fit_line(x):
    #     return slope * x + intercept
    # # Generate x values for the best fit line
    # x_fit = np.linspace(min(x_val), max(x_val), 100)  # Adjust the range of x_fit as needed
    # # Plot the best fit line
    # plt.plot(x_fit, best_fit_line(x_fit), color='red')
    # plt.title("Scatter Plot of Maximum x-values against Maximum Voltage with Best Fit Line ")
    # plt.show()


In [ ]:
#direct final maximum data and position plot
read_and_process_final_datasets("directData.csv")

In [ ]:
#guassian final maximum data and position plot
read_and_process_final_datasets("gaussianData.csv")